# Lab 4:  Cleaning data

We will look at a number of techniques for cleaning data in this lab.  We saw some of them previously in MAT 128.  This lab is set up to be used as a reference when cleaning data in the future.

We will use a made-up data file containing a number of common problems.  The made-up data has information about employees at a company, and the following columns:
* Name: first name of the employee
* Gender: gender of the employee
* Pay: yearly pay of the employee
* Months at job: number of months the employee has worked at the company

Download this made-up data from [https://raw.githubusercontent.com/megan-owen/MAT328-Techniques_in_Data_Science/main/data/Lab4_made-up_data.txt](https://raw.githubusercontent.com/megan-owen/MAT328-Techniques_in_Data_Science/main/data/Lab4_made-up_data.txt) and save it as a .txt file.

### Section 1:  Cleaning while loading the data file

First import the necessary libraries.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

We will assume the data is in a text file (a CSV file is a special kind of text file) and in rectangular format.  If the data is in a different format (ex. a PDF file), there are sometimes ways to extract it into a CSV file, but they are beyond the scope of this course.

There are 4 problems that can be solved as we read in the data file:
* extra rows at the beginning of the file
* extra rows at the end of the file
* columns delimited (separated) by something other than a comma.  Besides with commas, columns can also be separated by tabs, other white spaces, semi-colons (;), pipes (|), etc.
* missing data specified by something other than NaN

Open the data file `Lab4_made-up_data.txt` by clicking on it in Jupyter Hub, or opening it in a text editor like NotePad (Windows) or TextEdit (Mac).  Which of the above problems does this file have?

We can use the following parameters with `read_csv()`:

|Parameter|Example(s)|Effect of example|
|:-----|:-------------|:----------------|
|`skiprows`|`skiprows = 5`|Skips the first 5 rows in the file|
|`skipfooter`|`skipfooter = 3`|Skips the last 3 rows in the file|
|`na_values`|`na_values = "*"`|Treats \* as missing data|
||`na_values = ["-","."]`|Treats both - and . as missing data|
|`sep`|`sep = "\t"`|Columns are separated by tabs|
||`sep = "\s+"`|Columns are separated by any whitespaces (tabs, spaces, etc.)
||`sep = ";"`|Columns are separated by semi-colons|

For even more parameters that can be used with `read_csv()` see its [API](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html), which is a reference stating exactly how a function can be used.

We will actually need all of these parameters to clean up our data file!  Try using them with `read_csv` below to read in and clean the file:

Ignore the warning that happens.  Our command was:

`df = pd.read_csv("Lab4_made-up_data.txt", skiprows = 2, skipfooter = 2, na_values = "-", sep = "\t")`

and we skipped the first and last two rows in the data file, and indicated that `-` represented missing data and the columns are separated by tabs.  

Note that we have to use `sep = "\t"` instead of `sep = "\s+"`, since `sep = "\s+"` interprets the column name `Months at job` as the names for three separate columns (`Months`,`at`,`job`) due to the spaces.

If you haven't already, display your dataframe to see what else needs to be fixed and to check that `read_csv()` worked as expected.

### Section 2: Renaming columns

Sometimes column names are really long or otherwise unsuitable, and we want to rename them.  Here are two ways to rename the column `Months at job` to just `Months`.

**Method 1:** make a list of the columns names you want to use in the order they appear in the DataFrame, and assign it to `name_of_dataframe.columns`.

In [ ]:
new_columns = ["Name","Gender","Pay","Months"]
df.columns = new_columns

Display your dataframe to see if method 1 worked.

Reload your DataFrame from the file and try method 2.

**Method 2:** specify which column(s) to rename using what's called a dictionary (in {} brackets)

In [ ]:
df.rename(columns = {"Months at job":"Months"}, inplace = True)

Display your DataFrame again to check that method 2 works.

For method 2, you can change more than one column at a time like this:

`df.rename(columns = {"Months at job":"Months", "Pay": "Salary"}, inplace = True)`

### Section 3:  Remove duplicate rows 

Sometimes data sets have duplicate rows that we want to remove.  We will only look at the case where the entire row is a duplicate, but it is also possible to look for duplicates in a single column.

First, find the duplicated rows:

In [ ]:
dup_filter = df.duplicated()
df[dup_filter]

To drop the duplicates:

In [ ]:
df = df.drop_duplicates()
df

`df.drop_duplicates()` creates a new DataFrame without the duplicates, and we replace the original DataFrame `df` with this new one.

### Section 4: Changing the text in columns

Sometimes we want to fix the text in a column.  We look at two common examples in this DataFrame, but there are many other situations we are not covering.

#### 4.1 Standardize categories

In the `Gender` column, `male` has been represented by both `M` and `male`, while `female` has been represented by `F`, `f`, and `female`.  As humans, we can tell there are really only two categories, but the computer will assume there are 5 categories.

We need to standardize how `male` and `female` are stored in the DataFrame.  We do this by defining a *dictionary* that links the different variations to the category name we want to use, and then applies this *map* to the column.

So below, the dictionary `gender_map` will replace `M` and `male` with `male`, and `F`, `f`, and `female` with `female`.

In [ ]:
gender_map = {"M":"male", "F":"female", "f":"female", "male":"male", "female":"female"}
df["Gender"] = df["Gender"].map(gender_map)
df

#### 4.2 Removing characters and symbols

If a column in a DataFrame only contains numbers, the data values will be stored as `int` (integer) or `float` (number with decimals) and we can do computations with them.  However, if the column contains any letter or other character, all data values will be stored as `string`. 

In general, a *string* in Python (and other programming languages) is a sequence of letters, numbers, and characters.  Strings are often written with quotes around them.  For example, `"female"` and `"$57,000"` are both strings.  Python will NOT do computations with strings, even if we would as humans (for example, with the `pay` column).

Thus, we have to remove the dollar signs and commas in the pay column to be able to use this column as quantitative data.

We will use the `.replace(old,new)` function, which replaces the string `old` with the string `new`.  To delete something entirely, we replace it with the *empty string* `""`.

In [ ]:
df["Pay"] = df["Pay"].str.replace("$","")
df

What did the code above delete?  Next remove the commas in the `Pay` column.

### Section 5:  Changing column type

Unfortunately, we are not finished yet, because the `Pay` column is still stored as strings.  Check what types the columns are: 

<details><summary>Answer:</summary>
<code>df.dtypes</code>
</details>

What type is shown for the string columns?

Object is a catch-all type that includes string.

We need to convert the `Pay` column into type `int` using the column function `.astype(int)`.  Can you figure out how to do this?

<details><summary>Answer:</summary>
<code>df["Pay"] = df["Pay"].astype(int)</code>
</details>

Display the column types again to see if the `Pay` column changed.

We have now cleaned the data set, except for dealing with the missing data which we discussed last lab.